In [1]:
# Import dependencies
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine

In [2]:
#Import functions
def model(df, position, var):
    df = df.dropna()
    df = df[df['pos'] == position]   

    # the model will use different independent variables depending on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return

    y = df['next_yr_ppg']
    reg = linear_model.LinearRegression()
    reg.fit(X, y)
    return reg

def testModelAccuracy(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return

    yTest = df['next_yr_ppg']
    results = model.score(XTest, yTest)
    return results

def testModelDifference(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return


    yPred = model.predict(XTest)
    predAndActual = {'Name': df['player'], 'Predicted PPG': yPred,
                     'Actual PPG': df['next_yr_ppg']}

    # creates df from dictionary above
    database = pd.DataFrame(predAndActual)

    # creates a difference column which depicts the difference between the
    # predicted PPG and actual PPG
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals=3)
    database['Difference'] = database['Predicted PPG'] - database['Actual PPG']
    database['Difference'] = database['Difference'].round(decimals=3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)

    return database, meanDiff, medianDiff, meanAbsDiff, medianAbsDiff

def testModel(model, test, train, position, var):
    accuracy = testModelAccuracy(model, train, position, var)
    differences = testModelDifference(model, test, position, var)
    meanDiff = differences[1]
    medDiff = differences[2]
    meanAbsDiff = differences[3]
    medAbsDiff = differences[4]

    print('The accuracy of the {0} model is {1}'.format(position, accuracy))
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medDiff, medAbsDiff))
    print('\n')
    return

def checkModel(model, df1, df2, position, var):
    df1.dropna()
    df2.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df1 = df1[df1['pos'] == position]
    df2 = df2[df2['pos'] == position]
    df2 = df2[['plid','player','ppr_g']]
    newdf = df1.merge(df2,how='inner',left_on='plid',right_on='plid')

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df1[var]
    elif position == 'RB':
        X = df1[var]
    elif position == 'WR' or 'TE':
        X = df1[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df1['player'], 'Pos': df1['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.merge(df2,how='inner',left_on='Name',right_on='player')
    database = database.drop(columns=['player','plid'])
    database['Difference'] = database['Predicted PPG'] - database['ppr_g']
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medianDiff, medianAbsDiff))
    database = database.sort_values(by = ['ppr_g'], ascending = False)
    posRank = []
    posRankNum = 1
    for index, row in database.iterrows():
        posRank.append(posRankNum)
        posRankNum += 1

    database['ActPosRank'] = posRank
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
     # this adds a position rank column to the dataframe
    posRank = []
    posRankNum = 1
    for index, row in database.iterrows():
        posRank.append(posRankNum)
        posRankNum += 1

    database['PosRank'] = posRank
    

    database = database.reset_index(drop = True)

    return database

def finalPredict(model, df, position, var):
    df.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df = df[df['pos'] == position]

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df['player'], 'Pos': df['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)


     #this adds a position rank column to the dataframe
    posRank = []
    posRankNum = 1
    for index, row in df.iterrows():
        posRank.append(posRankNum)
        posRankNum += 1

    database['PosRank'] = posRank


    database = database.reset_index(drop = True)

    return database

def combineData(listOfDataFrames):
    # creates a df which contains the dataframes in the list which is passed in
    df = pd.concat(listOfDataFrames)
    df = df.sort_values(by = ['Predicted PPR'], ascending = False)

    # gives players an overall ranking
    rankings = []
    rank = 1
    for index, row in df.iterrows():
        rankings.append(rank)
        rank += 1

    df['Ovr Rank'] = rankings

    df = df.reset_index(drop = True)
    return df

In [3]:
# Connect to Database
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'fballfinalproject.c6sg90iemyn2.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'FFForesight5!!' ## CHANGE THIS TO YOUR POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Read in dataframes from AWS database
df = pd.read_sql_query('''SELECT *
FROM fantasy ;''',cnx)

starters = pd.read_sql_query('''SELECT starter AS qbstarter,
tm,
"Year",
plid AS stid,
tmid
FROM starters;''',cnx)

In [5]:
df = df.sort_values(by = ['player','yr'], ascending = [True,False])

In [6]:
# Add next year team to each player
nextYearTm = []
nextYearTmID = []
lastPlayer = 'NaN'
lastPlayerTm = 'NaN'


for index, row in df.iterrows():
    player = row['player']
    if lastPlayer == player:
        nextYearTm.append(lastPlayerTm)
        yr = row['yr'] + 1
        nextYearTmID.append(lastPlayerTm + str(yr))
    else:
        nextYearTm.append('NaN')
        nextYearTmID.append('NaN')
    lastPlayer = row['player']
    lastPlayerTm = row['tm']

df['NextTm'] = nextYearTm
df['NextTmID'] = nextYearTmID

In [7]:
# Merge with starter dataframe and cleanup
import numpy as np
df = pd.merge(df,starters[['tmid','qbstarter','stid']],left_on='NextTmID',right_on='tmid',how='outer')
df = df[df['player'].notna()]
df.drop(columns='tmid_y',inplace=True)
df.rename(columns={"tmid_x": "tmid"},inplace=True)
df['age'] = df['age'].apply(np.int64)
df['G'] = df['G'].apply(np.int64)
df['gs'] = df['gs'].apply(np.int64)
df['posrk'] = df['posrk'].apply(np.int64)
df['ovrank'] = df['ovrank'].apply(np.int64)
df['yr'] = df['yr'].apply(np.int64)
df['starter'] = df['starter'].apply(np.int64)

In [8]:
df = df.sort_values(by = ['player','yr'], ascending = [True,False],ignore_index=True)

In [9]:
start22 = starters[starters['Year'] == 2022]

In [10]:
# Get list of 2022 starters to add to rows
starters2022 = []
for index, row in start22.iterrows():
    starters2022.append(row['qbstarter'])

In [11]:
# Designate if QB is starting in 2022
for index, row in df.iterrows():
    if (row['yr'] == 2021) & (row['pos'] == 'QB'):
        if row['player'] in starters2022:
            df.at[index,'next_yr_starter'] = 1
        else:
            df.at[index,'next_yr_starter'] = 0
    else:
        continue

In [12]:
# List out df columns
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'NextTm', 'NextTmID', 'qbstarter', 'stid'],
      dtype='object')

In [13]:
# Select columns for each position (WR + TE joined in recvar)
qbvar = ['passatt_g','passyds_g','passtd_g','rushatt_g','rushyds_g',
         'rushyds_att','ppr_g','starter','next_yr_starter']
rbvar = ['age','rushatt_g','rushyds_g','rushyds_att','tgt_g','rec_g',
         'recyds_g','yds_rec','ppr_g']
recvar = ['age','tgt_g','rec_g','recyds_g','yds_rec','ppr_g']

In [14]:
# Separate into training, testing, using, and actual data by Yr (Fantasy) or Year (other dataframes)
train = df[df['yr'] <= 2020]
test = df[(df['yr'] >= 2020) & (df['yr'] <= 2021)]
use = df[df['yr'] == 2020]
use = use[use['next_yr_starter'].notna()]
use = use.astype({"next_yr_starter": int})
actuals = df[df['yr'] == 2021]

In [15]:
# Name model and put in position
QBmodel = model(train, "QB", qbvar)
RBmodel = model(train, "RB", rbvar)
WRmodel = model(train, "WR", recvar)
TEmodel = model(train, "TE", recvar)
testModel(QBmodel, test, train, "QB", qbvar)
testModel(RBmodel, test, train, "RB", rbvar)
testModel(WRmodel, test, train, "WR", recvar)
testModel(TEmodel, test, train, "TE", recvar)

The accuracy of the QB model is 0.6930395181025147
The QB model has an average error of 0.128 PPG and an average absolute error of 3.032 PPG
The QB model has a median error of 0.198 PPG and a median absolute error of 2.88 PPG


The accuracy of the RB model is 0.5338014291304849
The RB model has an average error of -0.105 PPG and an average absolute error of 2.718 PPG
The RB model has a median error of 0.193 PPG and a median absolute error of 1.947 PPG


The accuracy of the WR model is 0.5777329466547421
The WR model has an average error of 0.33 PPG and an average absolute error of 2.668 PPG
The WR model has a median error of 0.72 PPG and a median absolute error of 2.126 PPG


The accuracy of the TE model is 0.5348737476711556
The TE model has an average error of 0.158 PPG and an average absolute error of 1.954 PPG
The TE model has a median error of 0.79 PPG and a median absolute error of 1.462 PPG




In [16]:
# Use model to make predictions and check predictions
QBStats = checkModel(QBmodel, use, actuals, 'QB', qbvar)
RBStats = checkModel(RBmodel, use, actuals,'RB', rbvar)
WRStats = checkModel(WRmodel, use, actuals, 'WR', recvar)
TEStats = checkModel(TEmodel, use, actuals, 'TE', recvar)

The QB model has an average error of 0.127 PPG and an average absolute error of 3.032 PPG
The QB model has a median error of 0.198 PPG and a median absolute error of 2.88 PPG
The RB model has an average error of -0.019 PPG and an average absolute error of 2.721 PPG
The RB model has a median error of 0.316 PPG and a median absolute error of 2.069 PPG
The WR model has an average error of 0.524 PPG and an average absolute error of 2.633 PPG
The WR model has a median error of 0.812 PPG and a median absolute error of 2.14 PPG
The TE model has an average error of 0.131 PPG and an average absolute error of 1.983 PPG
The TE model has a median error of 0.79 PPG and a median absolute error of 1.482 PPG


In [17]:
data = [QBStats, RBStats, WRStats, TEStats]
fullData = combineData(data)
#fullData.to_csv('predictedAdvStats2021.csv')

In [18]:
fullData

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference,ActPosRank,PosRank,Ovr Rank
0,JoshAllen,QB,20.235,23.682,-3.447166,343.995,3.447166,1,1,1
1,DavanteAdams,WR,19.890,21.519,-1.629405,338.130,1.629405,2,1,2
2,DakPrescott,QB,19.705,20.038,-0.333340,334.985,0.333340,8,2,3
3,ChristianMcCaffrey,RB,19.399,18.214,1.184711,329.783,1.184711,5,1,4
4,KylerMurray,QB,18.836,21.464,-2.627723,320.212,2.627723,4,3,5
...,...,...,...,...,...,...,...,...,...,...
431,AndyJanovich,RB,1.227,0.915,0.312444,20.859,0.312444,108,124,432
432,DerekCarrier,TE,1.025,0.660,0.365493,17.425,0.365493,70,82,433
433,KendallHinton,WR,0.890,2.406,-1.516282,15.130,1.516282,126,167,434
434,NickBellore,RB,0.559,0.082,0.476938,9.503,0.476938,124,125,435


In [19]:
#QBSfinal = finalPredict(QBmodel,actuals, 'QB', qbvar1)
QBSfinal = finalPredict(QBmodel, actuals, 'QB', qbvar)
RBSfinal = finalPredict(RBmodel, actuals,'RB', rbvar)
WRSfinal = finalPredict(WRmodel, actuals, 'WR', recvar)
TESfinal = finalPredict(TEmodel, actuals, 'TE', recvar)

In [20]:
data = [QBSfinal,RBSfinal, WRSfinal, TESfinal]
fullData = combineData(data)
#fullData.to_csv('predictedStats2022.csv')

In [21]:
fullData

,Name,Pos,Predicted PPG,Predicted PPR,PosRank,Ovr Rank
0,CooperKupp,WR,20.806,353.702,1,1
1,JalenHurts,QB,19.157,325.669,1,2
2,KylerMurray,QB,18.895,321.215,2,3
3,LamarJackson,QB,18.286,310.862,3,4
4,JoshAllen,QB,18.165,308.805,4,5
...,...,...,...,...,...,...
589,AndyJanovich,RB,0.980,16.660,164,590
590,TrentonCannon,RB,0.976,16.592,165,591
591,DerekWatt,RB,0.637,10.829,166,592
592,AndreRoberts,WR,-0.066,-1.122,227,593
